In [2]:
import pandas as pd
import numpy as np
import scilearn as skt

ModuleNotFoundError: No module named 'scilearn'

In [3]:
df = pd.read_csv("..\Corpora\ICE INDIA\Corpus\S1a-001.txt", sep="<$A>|<$B>", engine='python')


In [30]:
pip install -U scikit-learn

     ---------------------------------------- 8.3/8.3 MB 433.4 kB/s eta 0:00:00
     ------------------------------------ 298.0/298.0 kB 735.9 kB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Note: you may need to restart the kernel to use updated packages.


In [5]:
df = df.replace(to_replace=r'<.*>', value='', regex=True)
print(df)

                           <I>
0                             
1                             
2    May I know your good name
3                             
4                             
..                         ...
711                           
712                 Thank you 
713                           
714                           
715       Thank you very much 

[716 rows x 1 columns]


In [6]:
padded_numbers = [number.zfill(3) for number in numbers]
print(padd_numbers)

NameError: name 'numbers' is not defined

In [7]:

count = [100, 80, 70, 50]
prefix = ["S1A", "S1B", "S2A", "S2B"]
for i in range(0, 4):
    end=count[i]+1
    active_prefix = prefix[i]
    for i in range(1, end):
        combined_string=''
        index_padded = str(i).zfill(3)
        filepath = "..\Corpora\ICE INDIA\Corpus\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df = pd.read_csv(filepath, sep="<$A>|<$B>", engine='python', encoding='ISO-8859-1')
        temp_df = temp_df.replace(to_replace="\[[^\[\]]*\]", value='', regex=True)
        #combined_string = ' '.join(temp_df.apply(lambda x: ' '.join(x), axis=1))
        new_filepath = "..\CleanCorpora\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df.to_csv(new_filepath, sep='\t', index=False)
        
        
#print(combined_string)
    

In [6]:
count = [100, 80, 70, 50]
prefix = ["S1A", "S1B", "S2A", "S2B"]
comb = pd.DataFrame()
for i in range(0, 4):
    end=count[i]+1
    active_prefix = prefix[i]
    for i in range(1, end):
        index_padded = str(i).zfill(3)
        filepath = "..\Corpora\ICE INDIA\Corpus\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df = pd.read_csv(filepath, sep="<$A>|<$B>", engine='python', encoding='ISO-8859-1')
        temp_df = temp_df.replace(to_replace=r'<[^<>]*>', value='', regex=True)
        temp_df = temp_df.replace("", np.nan)
        temp_df = temp_df.replace("\s+", " ", regex=True).dropna()
        comb = comb.append(temp_df)
        new_filepath = "..\CleanCorpora\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df.to_csv(new_filepath, sep='\t', index=False)
comb_filepath = "..\CleanCorpora\_IndianCorporaCombined.txt"
comb.to_csv(comb_filepath, sep='\t', index=False)

C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning

C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning

C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning

C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning

C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning

C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning

C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb = comb.append(temp_df)
C:\Users\iwanr\AppData\Local\Temp\ipykernel_2212\2223088058.py:14: FutureWarning

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from io import open

In [8]:
# Open the text file
with open("..\CleanCorpora\_IndianCorporaCombined.txt", "r", encoding='ISO-8859-1') as file:
    text = file.read()
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([text])
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x, twenty_train.target)

In [9]:
print(X)

  (0, 10296)	1.6536507548014483e-05
  (0, 8595)	1.6536507548014483e-05
  (0, 20710)	1.6536507548014483e-05
  (0, 12335)	1.6536507548014483e-05
  (0, 12337)	1.6536507548014483e-05
  (0, 8220)	1.6536507548014483e-05
  (0, 22844)	4.9609522644043446e-05
  (0, 22843)	1.6536507548014483e-05
  (0, 8101)	1.6536507548014483e-05
  (0, 18559)	1.6536507548014483e-05
  (0, 779)	1.6536507548014483e-05
  (0, 17420)	1.6536507548014483e-05
  (0, 9571)	6.614603019205793e-05
  (0, 6107)	1.6536507548014483e-05
  (0, 3138)	1.6536507548014483e-05
  (0, 9652)	4.9609522644043446e-05
  (0, 8300)	1.6536507548014483e-05
  (0, 22793)	1.6536507548014483e-05
  (0, 11381)	1.6536507548014483e-05
  (0, 23472)	1.6536507548014483e-05
  (0, 19864)	1.6536507548014483e-05
  (0, 14160)	1.6536507548014483e-05
  (0, 19863)	1.6536507548014483e-05
  (0, 7180)	1.6536507548014483e-05
  (0, 15197)	1.6536507548014483e-05
  :	:
  (0, 23239)	0.02910425328450549
  (0, 14577)	0.08764349000447676
  (0, 14518)	0.28470905045416534
  (0, 4